In [ ]:
import pygame
import math
import random
import time
import sys
import os
from PIL import Image, ImageSequence

# ตั้งค่า Pygame และ Pygame Mixer
pygame.init()
pygame.mixer.init()

# ความกว้างและความสูงของหน้าจอ
WIDTH, HEIGHT = 800, 600

# สี
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
GRAY = (150, 150, 150)
DARK_GREEN = (0, 200, 0)
DARK_GRAY = (100, 100, 100)
LIGHT_GRAY = (200, 200, 200)
DARKER_GRAY = (80, 80, 80)
RED = (255, 0, 0)
DARK_RED = (200, 0, 0)
ORANGE = (255, 165, 0)
DARK_ORANGE = (200, 130, 0)

# คลาสและฟังก์ชันจาก economics_game_logic.py
class Player:
    def __init__(self, name):
        self.name = name
        self.money = 1000
        self.inventory = {"rice": 0, "water": 0, "tools": 0}
        self.day = 1

    def buy(self, item, amount, market):
        cost = market.prices[item] * amount
        if amount <= 0:
            return "จำนวนต้องมากกว่า 0!"
        elif self.money >= cost and market.supply[item] >= amount:
            self.money -= cost
            self.inventory[item] += amount
            market.supply[item] -= amount
            return f"ซื้อ {item} {amount} หน่วยสำเร็จ!"
        return "เงินไม่พอหรือสินค้าในตลาดไม่เพียง!"

    def sell(self, item, amount, market):
        if amount <= 0:
            return "จำนวนต้องมากกว่า 0!"
        elif self.inventory[item] >= amount:
            self.money += market.prices[item] * amount
            self.inventory[item] -= amount
            market.supply[item] += amount
            return f"ขาย {item} {amount} หน่วยสำเร็จ!"
        return "สินค้าในคลังไม่พอ!"

class Market:
    def __init__(self):
        self.prices = {"rice": 20, "water": 10, "tools": 50}
        self.supply = {"rice": 100, "water": 150, "tools": 50}

    def adjust_prices(self):
        demand = {
            "rice": random.randint(50, 150),
            "water": random.randint(80, 200),
            "tools": random.randint(20, 80)
        }
        if demand["rice"] > self.supply["rice"]:
            self.prices["rice"] += 5
        elif demand["rice"] < self.supply["rice"]:
            self.prices["rice"] = max(5, self.prices["rice"] - 2)

        if demand["water"] > self.supply["water"]:
            self.prices["water"] += 3
        elif demand["water"] < self.supply["water"]:
            self.prices["water"] = max(3, self.prices["water"] - 1)

        if demand["tools"] > self.supply["tools"]:
            self.prices["tools"] += 10
        elif demand["tools"] < self.supply["tools"]:
            self.prices["tools"] = max(20, self.prices["tools"] - 5)

class Game:
    def __init__(self, player_name):
        self.player = Player(player_name)
        self.market = Market()
        self.message = "เริ่มเกม!"

    def next_day(self):
        self.market.adjust_prices()
        self.player.day += 1
        self.message = "วันใหม่เริ่มแล้ว!"

    def check_game_over(self):
        if self.player.money <= 0:
            return "แพ้: เงินหมด!"
        elif self.player.day > 10:
            if self.player.money >= 1500:
                return f"ชนะ: คุณมี {self.player.money} บาท!"
            return f"แพ้: หมดวัน มีเงิน {self.player.money} บาท ไม่ถึงเป้า!"
        return None

    def get_player_data(self):
        return {
            "day": self.player.day,
            "money": self.player.money,
            "inventory": self.player.inventory
        }

    def get_market_data(self):
        return {
            "prices": self.market.prices
        }

# ฟังก์ชันจาก economics_game_menu.py
def get_rainbow_color(time_offset):
    r = int(math.sin(time_offset + 0) * 127 + 128)
    g = int(math.sin(time_offset + 2) * 127 + 128)
    b = int(math.sin(time_offset + 4) * 127 + 128)
    return (r, g, b)

def draw_text(screen, text, x, y, font_size=30, color=BLACK, font_path="THSarabunNew.ttf", rotate_angle=0, outline=False, shadow=False, outline_color=WHITE, center=False):
    if isinstance(font_size, tuple):
        font_size = font_size[0]
    font = pygame.font.Font(font_path, font_size)
    text_surface = font.render(text, True, color)

    if rotate_angle != 0:
        text_surface = pygame.transform.rotate(text_surface, rotate_angle)

    if "Bold" not in font_path:
        temp_surface = pygame.Surface((text_surface.get_width() + 2, text_surface.get_height() + 2), pygame.SRCALPHA)
        for dx, dy in [(0, 0), (1, 0), (0, 1), (1, 1)]:
            temp_surface.blit(font.render(text, True, color), (dx, dy))
        if rotate_angle != 0:
            temp_surface = pygame.transform.rotate(temp_surface, rotate_angle)
        text_surface = temp_surface

    if outline:
        outline_surface = pygame.Surface((text_surface.get_width() + 4, text_surface.get_height() + 4), pygame.SRCALPHA)
        for offset in [(-1, -1), (-1, 1), (1, -1), (1, 1), (0, -1), (0, 1), (-1, 0), (1, 0)]:
            outline_surface.blit(font.render(text, True, outline_color), (offset[0] + 2, offset[1] + 2))
        outline_surface.blit(text_surface, (2, 2))
        text_surface = outline_surface

    if shadow:
        shadow_surface = pygame.Surface((text_surface.get_width() + 2, text_surface.get_height() + 2), pygame.SRCALPHA)
        shadow_surface.blit(font.render(text, True, (0, 0, 0, 100)), (2, 2))
        if rotate_angle != 0:
            shadow_surface = pygame.transform.rotate(shadow_surface, rotate_angle)
        screen.blit(shadow_surface, (x + 2 - shadow_surface.get_width() // 2, y + 2 - shadow_surface.get_height() // 2))

    rect = text_surface.get_rect()
    if center:
        final_x = x - rect.width // 2
        final_y = y - rect.height // 2
    else:
        final_x = x
        final_y = y
    screen.blit(text_surface, (final_x, final_y))
    return rect

def load_gif(gif_path):
    try:
        gif = Image.open(gif_path)
        frames = []
        for frame in ImageSequence.Iterator(gif):
            frame = frame.convert('RGBA')
            mode = frame.mode
            size = frame.size
            data = frame.tobytes()
            pygame_image = pygame.image.fromstring(data, size, mode)
            pygame_image = pygame.transform.scale(pygame_image, (800, 600))
            frames.append(pygame_image)
        return frames, gif.info['duration']
    except Exception as e:
        frame = pygame.Surface((800, 600), pygame.SRCALPHA)
        frame.fill((173, 216, 230))
        return [frame], 100

def draw_menu(screen, gif_frames, frame_duration, frame_index, last_frame_time, button_states=None, click_sound=None):
    if button_states is None:
        button_states = {"start": False, "instructions": False, "developers": False}

    current_time = pygame.time.get_ticks()
    if current_time - last_frame_time > frame_duration:
        frame_index = (frame_index + 1) % len(gif_frames)
        last_frame_time = current_time

    screen.blit(gif_frames[frame_index], (0, 0))

    welcome_text = "ยินดีต้อนรับสู่ตลาดนักเศรษฐศาสตร์รุ่นใหม่"
    draw_text(screen, welcome_text, 800 // 2, 100, font_size=40, color=BLACK, font_path="THSarabunNew.ttf",
              outline=True, shadow=True, outline_color=WHITE, center=True)

    buttons = [
        {"text": "เริ่มเกม", "rect": pygame.Rect(300, 250, 200, 50), "color": GREEN, "action": "start", "font_size": 30},
        {"text": "คำอธิบายเกม", "rect": pygame.Rect(300, 350, 200, 50), "color": GRAY, "action": "instructions", "font_size": 30},
        {"text": "สมาชิกในกลุ่ม", "rect": pygame.Rect(800 // 2 - 100, 450, 200, 50), "color": GRAY, "action": "developers", "font_size": 30}
    ]

    mouse_pos = pygame.mouse.get_pos()
    for btn in buttons:
        action = btn["action"]
        clicked = btn["rect"].collidepoint(mouse_pos) and pygame.mouse.get_pressed()[0]
        hover = btn["rect"].collidepoint(mouse_pos) and not clicked

        if hover and not button_states[action]:
            if click_sound:
                click_sound.play()
            button_states[action] = True
        elif not hover:
            button_states[action] = False

        base_color = btn["color"]
        current_color = DARK_GREEN if clicked and btn["color"] == GREEN else DARK_GRAY if clicked else base_color

        pygame.draw.rect(screen, current_color, btn["rect"])
        pygame.draw.line(screen, LIGHT_GRAY, (btn["rect"].left, btn["rect"].top), (btn["rect"].right - 1, btn["rect"].top), 2)
        pygame.draw.line(screen, LIGHT_GRAY, (btn["rect"].left, btn["rect"].top), (btn["rect"].left, btn["rect"].bottom - 1), 2)
        pygame.draw.line(screen, DARKER_GRAY, (btn["rect"].right - 1, btn["rect"].top), (btn["rect"].right - 1, btn["rect"].bottom - 1), 2)
        pygame.draw.line(screen, DARKER_GRAY, (btn["rect"].left, btn["rect"].bottom - 1), (btn["rect"].right - 1, btn["rect"].bottom - 1), 2)

        if hover:
            pygame.draw.rect(screen, WHITE, btn["rect"], 2)

        text_color = WHITE if hover else BLACK
        rotate_angle = 5 if hover else 0
        draw_text(screen, btn["text"], btn["rect"].centerx, btn["rect"].centery, font_size=btn["font_size"], color=text_color, rotate_angle=rotate_angle, center=True)

    pygame.display.flip()
    return buttons, frame_index, last_frame_time, button_states

def draw_instructions(screen, gif_frames, frame_index, frame_duration, last_frame_time, back_button_state=False, click_sound=None):
    current_time = pygame.time.get_ticks()
    if current_time - last_frame_time > frame_duration:
        frame_index = (frame_index + 1) % len(gif_frames)
        last_frame_time = current_time

    screen.blit(gif_frames[frame_index], (0, 0))

    title_font = pygame.font.Font("THSarabunNew.ttf", 40)
    title_surface = title_font.render("คำอธิบายเกม", True, BLACK)
    temp_surface = pygame.Surface((title_surface.get_width() + 2, title_surface.get_height() + 2), pygame.SRCALPHA)
    for dx, dy in [(0, 0), (1, 0), (0, 1), (1, 1)]:
        temp_surface.blit(title_font.render("คำอธิบายเกม", True, BLACK), (dx, dy))
    title_surface = temp_surface
    title_rect = title_surface.get_rect(center=(800 // 2, 80))
    screen.blit(title_surface, title_rect.topleft)

    instructions = [
        "1. ซื้อขายสินค้าในตลาดเสมือนจริง",
        "2. บริหารเงินทุนให้เติบโต",
        "3. เรียนรู้หลักเศรษฐศาสตร์ผ่านการเล่น"
    ]

    instruction_font = pygame.font.Font("THSarabunNew.ttf", 30)
    max_width = 0
    line_height = 0
    y_positions = []
    for i, line in enumerate(instructions):
        y_position = 150 + i * 60
        y_positions.append(y_position)
        rect = instruction_font.render(line, True, BLACK).get_rect()
        max_width = max(max_width, rect.width)
        line_height = rect.height

    padding_x = 20
    padding_y = 15
    box_width = max_width + padding_x * 2
    box_height = (line_height * 3) + (60 * 2) + (padding_y * 2)
    box_x = (800 - box_width) // 2
    box_y = 150 - padding_y - 10

    background_rect = pygame.Rect(box_x, box_y, box_width, box_height)
    pygame.draw.rect(screen, WHITE, background_rect)
    for i in range(3):
        for x in range(background_rect.left, background_rect.right):
            color = get_rainbow_color(current_time / 1000 + x / 100)
            pygame.draw.line(screen, color, (x, background_rect.top + i), (x, background_rect.top + i))
            pygame.draw.line(screen, color, (x, background_rect.bottom - 1 - i), (x, background_rect.bottom - 1 - i))
        for y in range(background_rect.top, background_rect.bottom):
            color = get_rainbow_color(current_time / 1000 + y / 100)
            pygame.draw.line(screen, color, (background_rect.left + i, y), (background_rect.left + i, y))
            pygame.draw.line(screen, color, (background_rect.right - 1 - i, y), (background_rect.right - 1 - i, y))

    for i, line in enumerate(instructions):
        text_surface = instruction_font.render(line, True, BLACK)
        text_rect = text_surface.get_rect(topleft=(box_x + padding_x, y_positions[i] - line_height // 2))
        screen.blit(text_surface, text_rect.topleft)

    back_button = pygame.Rect(50, 500, 200, 50)
    pygame.draw.rect(screen, GRAY, back_button)
    pygame.draw.line(screen, LIGHT_GRAY, (back_button.left, back_button.top), (back_button.right - 1, back_button.top), 2)
    pygame.draw.line(screen, LIGHT_GRAY, (back_button.left, back_button.top), (back_button.left, back_button.bottom - 1), 2)
    pygame.draw.line(screen, DARK_GRAY, (back_button.right - 1, back_button.top), (back_button.right - 1, back_button.bottom - 1), 2)
    pygame.draw.line(screen, DARK_GRAY, (back_button.left, back_button.bottom - 1), (back_button.right - 1, back_button.bottom - 1), 2)

    mouse_pos = pygame.mouse.get_pos()
    clicked = back_button.collidepoint(mouse_pos) and pygame.mouse.get_pressed()[0]
    hover = back_button.collidepoint(mouse_pos) and not clicked

    if hover and not back_button_state:
        if click_sound:
            click_sound.play()
        back_button_state = True
    elif not hover:
        back_button_state = False

    if hover:
        pygame.draw.rect(screen, WHITE, back_button, 2)

    text_color = WHITE if hover else BLACK
    rotate_angle = 5 if hover else 0
    draw_text(screen, "ย้อนกลับ", back_button.centerx, back_button.centery, font_size=30, color=text_color, rotate_angle=rotate_angle, center=True)

    pygame.display.flip()
    return back_button, frame_index, last_frame_time, back_button_state

def draw_developers(screen, gif_frames, frame_index, frame_duration, last_frame_time, back_button_state=False, click_sound=None):
    current_time = pygame.time.get_ticks()
    if current_time - last_frame_time > frame_duration:
        frame_index = (frame_index + 1) % len(gif_frames)
        last_frame_time = current_time

    screen.blit(gif_frames[frame_index], (0, 0))

    title_font = pygame.font.Font("THSarabunNew.ttf", 40)
    title_surface = title_font.render("สมาชิกในกลุ่ม", True, BLACK)
    temp_surface = pygame.Surface((title_surface.get_width() + 2, title_surface.get_height() + 2), pygame.SRCALPHA)
    for dx, dy in [(0, 0), (1, 0), (0, 1), (1, 1)]:
        temp_surface.blit(title_font.render("สมาชิกในกลุ่ม", True, BLACK), (dx, dy))
    title_surface = temp_surface
    title_rect = title_surface.get_rect(center=(800 // 2, 80))
    screen.blit(title_surface, title_rect.topleft)

    members = [
        "1. นางสาวชนาภัทร โมฮารัม 006",
        "2. นายปฏิภาณ แก้ววิเศษ 013",
        "3. นางสาวสินีนาถ ขาวพรม 024",
        "4. นางสาว อภิชญา ไบลากวน 026",
        "5. นายจีฮาน เจ๊ะพงศ์ 034"
    ]

    member_font = pygame.font.Font("THSarabunNew.ttf", 30)
    for i, member in enumerate(members):
        member_surface = member_font.render(member, True, BLACK)
        member_rect = member_surface.get_rect(center=(800 // 2, 150 + i * 60))
        screen.blit(member_surface, member_rect.topleft)

    back_button = pygame.Rect(50, 500, 200, 50)
    pygame.draw.rect(screen, GRAY, back_button)
    pygame.draw.line(screen, LIGHT_GRAY, (back_button.left, back_button.top), (back_button.right - 1, back_button.top), 2)
    pygame.draw.line(screen, LIGHT_GRAY, (back_button.left, back_button.top), (back_button.left, back_button.bottom - 1), 2)
    pygame.draw.line(screen, DARK_GRAY, (back_button.right - 1, back_button.top), (back_button.right - 1, back_button.bottom - 1), 2)
    pygame.draw.line(screen, DARK_GRAY, (back_button.left, back_button.bottom - 1), (back_button.right - 1, back_button.bottom - 1), 2)

    mouse_pos = pygame.mouse.get_pos()
    clicked = back_button.collidepoint(mouse_pos) and pygame.mouse.get_pressed()[0]
    hover = back_button.collidepoint(mouse_pos) and not clicked

    if hover and not back_button_state:
        if click_sound:
            click_sound.play()
        back_button_state = True
    elif not hover:
        back_button_state = False

    if hover:
        pygame.draw.rect(screen, WHITE, back_button, 2)

    text_color = WHITE if hover else BLACK
    rotate_angle = 5 if hover else 0
    draw_text(screen, "ย้อนกลับ", back_button.centerx, back_button.centery, font_size=30, color=text_color, rotate_angle=rotate_angle, center=True)

    pygame.display.flip()
    return back_button, frame_index, last_frame_time, back_button_state

def main_menu(screen, click_sound=None):
    pygame.display.set_caption("ตลาดนักเศรษฐศาสตร์รุ่นใหม่")
    clock = pygame.time.Clock()

    try:
        pygame.mixer.music.load("background_music.mp3")
        pygame.mixer.music.set_volume(0.3)
        pygame.mixer.music.play(-1)
    except FileNotFoundError:
        pass

    gif_frames, frame_duration = load_gif("background.gif")
    frame_index = 0
    last_frame_time = pygame.time.get_ticks()

    running = True
    current_screen = "menu"
    buttons = None
    back_button = None
    button_states = None
    back_button_state = False

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.mixer.music.stop()
                pygame.quit()
                return False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                if current_screen == "menu":
                    for btn in buttons:
                        if btn["rect"].collidepoint(pos):
                            if btn["action"] == "start":
                                pygame.mixer.music.stop()
                                return True
                            elif btn["action"] == "instructions":
                                current_screen = "instructions"
                            elif btn["action"] == "developers":
                                current_screen = "developers"
                elif current_screen == "instructions":
                    if back_button and back_button.collidepoint(pos):
                        current_screen = "menu"
                elif current_screen == "developers":
                    if back_button.collidepoint(pos):
                        current_screen = "menu"
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE and (current_screen == "instructions" or current_screen == "developers"):
                    current_screen = "menu"

        if current_screen == "menu":
            buttons, frame_index, last_frame_time, button_states = draw_menu(screen, gif_frames, frame_duration, frame_index, last_frame_time, button_states, click_sound)
        elif current_screen == "instructions":
            back_button, frame_index, last_frame_time, back_button_state = draw_instructions(screen, gif_frames, frame_index, frame_duration, last_frame_time, back_button_state, click_sound)
        elif current_screen == "developers":
            back_button, frame_index, last_frame_time, back_button_state = draw_developers(screen, gif_frames, frame_index, frame_duration, last_frame_time, back_button_state, click_sound)

        clock.tick(60)

    pygame.mixer.music.stop()
    return True

# ฟังก์ชันจาก economics_game_play.py
def scale_image(image, target_width=150):
    original_width, original_height = image.get_size()
    aspect_ratio = original_height / original_width
    target_height = int(target_width * aspect_ratio)
    return pygame.transform.scale(image, (target_width, target_height))

def blur_surface(surface, factor=4):
    width, height = surface.get_size()
    small = pygame.transform.smoothscale(surface, (width // factor, height // factor))
    return pygame.transform.smoothscale(small, (width, height))

def draw_button(surface, rect, color, text, hover=False, clicked=False):
    base_color = color
    if clicked:
        if color == GREEN:
            current_color = DARK_GREEN
        elif color == RED:
            current_color = DARK_RED
        elif color == ORANGE:
            current_color = DARK_ORANGE
        else:
            current_color = DARK_GRAY
    else:
        current_color = base_color

    pygame.draw.rect(surface, current_color, rect)
    pygame.draw.line(surface, LIGHT_GRAY, (rect.left, rect.top), (rect.right - 1, rect.top), 2)
    pygame.draw.line(surface, LIGHT_GRAY, (rect.left, rect.top), (rect.left, rect.bottom - 1), 2)
    pygame.draw.line(surface, DARK_GRAY, (rect.right - 1, rect.top), (rect.right - 1, rect.bottom - 1), 2)
    pygame.draw.line(surface, DARK_GRAY, (rect.left, rect.bottom - 1), (rect.right - 1, rect.bottom - 1), 2)

    if hover:
        pygame.draw.rect(surface, WHITE, rect, 2)

    draw_text(surface, text, rect.centerx, rect.centery - 5, font_size=24, color=WHITE if hover else BLACK, center=True)

class Character:
    def __init__(self, x, y, character_front_image, character_left_image, character_right_image):
        self.rect = character_front_image.get_rect(center=(x, y))
        self.target = None
        self.speed = 5
        self.direction = "front"
        self.character_front_image = character_front_image
        self.character_left_image = character_left_image
        self.character_right_image = character_right_image

    def update(self, status_rect=None):
        if self.target:
            dx = self.target[0] - self.rect.centerx
            dy = self.target[1] - self.rect.centery
            distance = (dx ** 2 + dy ** 2) ** 0.5

            if dx > 0:
                self.direction = "right"
            elif dx < 0:
                self.direction = "left"
            else:
                self.direction = "front"

            if status_rect and distance < 20 and status_rect.colliderect(self.rect.move(dx, dy)):
                self.target = None
            elif distance > self.speed:
                angle = pygame.math.Vector2(dx, dy).normalize()
                new_x = self.rect.x + angle.x * self.speed
                new_y = self.rect.y + angle.y * self.speed
                new_rect = self.rect.move(new_x - self.rect.x, new_y - self.rect.y)
                if 0 <= new_rect.left and new_rect.right <= WIDTH and 0 <= new_rect.top and new_rect.bottom <= HEIGHT:
                    self.rect.x = new_x
                    self.rect.y = new_y
                else:
                    self.target = None
            else:
                self.rect.center = self.target
                self.target = None
                self.direction = "front"

    def draw(self, surface):
        if self.direction == "left":
            surface.blit(self.character_left_image, self.rect)
        elif self.direction == "right":
            surface.blit(self.character_right_image, self.rect)
        else:
            surface.blit(self.character_front_image, self.rect)

def draw_main_scene(screen, game, character, last_hovered_shop=None, fade_alpha=0, next_day_clicked=False, show_status=False, background_image=None, shop1_image=None, shop2_image=None, shop3_image=None, shop_sound=None, click_sound=None):
    screen.blit(background_image, (0, 0))

    draw_text(screen, game.message, WIDTH // 2, HEIGHT - 50, font_size=24, color=BLACK, center=True)

    shops = [
        {"name": "ร้านข้าว", "image": shop1_image, "rect": shop1_image.get_rect(center=(100, HEIGHT - 150)), "item": "rice", "number": 1},
        {"name": "ร้านน้ำ", "image": shop2_image, "rect": shop2_image.get_rect(center=(400, HEIGHT - 100)), "item": "water", "number": 2},
        {"name": "ร้านเครื่องมือ", "image": shop3_image, "rect": shop3_image.get_rect(center=(700, HEIGHT - 150)), "item": "tools", "number": 3},
    ]

    mouse_pos = pygame.mouse.get_pos()
    current_hovered_shop = None

    for shop in shops:
        screen.blit(shop["image"], shop["rect"])
        if shop["rect"].collidepoint(mouse_pos):
            current_hovered_shop = shop["name"]
            if shop_sound and current_hovered_shop != last_hovered_shop:
                shop_sound.play()
            text_surface = pygame.font.SysFont("THSarabunNew", 24).render(shop["name"], True, BLACK)
            bg_width = text_surface.get_width() + 10
            bg_height = text_surface.get_height() + 6
            bg_rect_shop = pygame.Rect(shop["rect"].centerx - bg_width // 2, shop["rect"].top - bg_height - 5, bg_width, bg_height)
            pygame.draw.rect(screen, WHITE, bg_rect_shop)
            pygame.draw.rect(screen, BLACK, bg_rect_shop, 2)
            draw_text(screen, shop["name"], shop["rect"].centerx, shop["rect"].top - bg_height // 2 - 5, font_size=24, color=BLACK, center=True)

    character.draw(screen)
    if character.rect.collidepoint(mouse_pos):
        character_name = "น้องลูกท่อง"
        text_surface = pygame.font.SysFont("THSarabunNew", 24).render(character_name, True, BLACK)
        bg_width = text_surface.get_width() + 10
        bg_height = text_surface.get_height() + 6
        bg_rect_char = pygame.Rect(character.rect.centerx - bg_width // 2, character.rect.top - bg_height - 5, bg_width, bg_height)
        pygame.draw.rect(screen, WHITE, bg_rect_char)
        pygame.draw.rect(screen, BLACK, bg_rect_char, 2)
        draw_text(screen, character_name, character.rect.centerx, character.rect.top - bg_height // 2 - 5, font_size=24, color=BLACK, center=True)

    next_day_button = {"rect": pygame.Rect(WIDTH - 150, 20, 130, 50), "color": ORANGE, "text": "วันถัดไป", "action": "next_day"}
    clicked = pygame.mouse.get_pressed()[0]
    hover = next_day_button["rect"].collidepoint(mouse_pos)

    if clicked and hover and not next_day_clicked:
        if click_sound:
            click_sound.play()
        next_day_clicked = True
    elif not clicked:
        next_day_clicked = False

    draw_button(screen, next_day_button["rect"], next_day_button["color"], next_day_button["text"], hover, clicked and hover)

    if fade_alpha > 0:
        fade_surface = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
        fade_surface.fill((255, 255, 255, fade_alpha))
        screen.blit(fade_surface, (0, 0))

    pygame.display.flip()
    return shops, next_day_button, current_hovered_shop, next_day_clicked

def draw_shop_menu(screen, game, shop_name, shop_item, shop_number, fade_alpha=0, button_clicked=None, background_shop1=None, background_shop2=None, background_shop3=None, click_sound=None):
    if button_clicked is None:
        button_clicked = {"buy": False, "sell": False, "exit": False}

    if shop_number == 1:
        screen.blit(background_shop1, (0, 0))
    elif shop_number == 2:
        screen.blit(background_shop2, (0, 0))
    elif shop_number == 3:
        screen.blit(background_shop3, (0, 0))

    market_data = game.get_market_data()

    menu_rect = pygame.Rect(150, 200, 500, 350)
    pygame.draw.rect(screen, WHITE, menu_rect)
    pygame.draw.rect(screen, GRAY, menu_rect, 3)

    draw_text(screen, f"ร้านค้า: {shop_name}", menu_rect.centerx, menu_rect.top + 20, font_size=24, color=BLACK, center=True)

    item_name = "ข้าว" if shop_item == "rice" else "น้ำ" if shop_item == "water" else "เครื่องมือ"
    unit = "กก." if shop_item == "rice" else "ขวด" if shop_item == "water" else "ชิ้น"
    draw_text(screen, f"ราคา: {item_name} {market_data['prices'][shop_item]} บาท/{unit}",
              menu_rect.centerx, menu_rect.top + 60, font_size=24, color=BLACK, center=True)
    draw_text(screen, f"สินค้าในตลาด: {game.market.supply[shop_item]} {unit}",
              menu_rect.centerx, menu_rect.top + 100, font_size=24, color=BLACK, center=True)

    buttons = [
        {"text": f"ซื้อ{item_name}", "rect": pygame.Rect(200, 400, 150, 50), "color": GREEN, "action": f"buy_{shop_item}", "key": "buy"},
        {"text": f"ขาย{item_name}", "rect": pygame.Rect(450, 400, 150, 50), "color": RED, "action": f"sell_{shop_item}", "key": "sell"},
        {"text": "ออกจากร้าน", "rect": pygame.Rect(325, 500, 150, 50), "color": GRAY, "action": "exit_shop", "key": "exit"},
    ]

    mouse_pos = pygame.mouse.get_pos()
    clicked = any(pygame.mouse.get_pressed()[0] for _ in range(1))

    for btn in buttons:
        hover = btn["rect"].collidepoint(mouse_pos)
        if clicked and hover and not button_clicked[btn["key"]]:
            if click_sound:
                click_sound.play()
            button_clicked[btn["key"]] = True
        elif not clicked:
            button_clicked[btn["key"]] = False
        draw_button(screen, btn["rect"], btn["color"], btn["text"], hover, clicked and hover)

    if fade_alpha > 0:
        fade_surface = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
        fade_surface.fill((255, 255, 255, fade_alpha))
        screen.blit(fade_surface, (0, 0))

    pygame.display.flip()
    return buttons, button_clicked

def draw_input_menu(screen, message, input_amount, input_active, fade_alpha=0, back_button_clicked=False, background_image=None, click_sound=None):
    screen.blit(background_image, (0, 0))

    menu_rect = pygame.Rect(250, 200, 300, 250)
    pygame.draw.rect(screen, WHITE, menu_rect)
    pygame.draw.rect(screen, GRAY, menu_rect, 3)

    draw_text(screen, message, menu_rect.centerx, menu_rect.top + 20, font_size=24, color=BLACK, center=True)

    input_rect = pygame.Rect(300, 300, 200, 50)
    border_color = GREEN if input_active else GRAY
    pygame.draw.rect(screen, LIGHT_GRAY, input_rect)
    pygame.draw.rect(screen, border_color, input_rect, 2)
    draw_text(screen, input_amount or "กรอกจำนวน...", input_rect.centerx, input_rect.centery - 5, font_size=24, color=BLACK, center=True)

    back_button = {"rect": pygame.Rect(300, 360, 200, 50), "color": GRAY, "text": "ย้อนกลับ", "action": "back"}
    mouse_pos = pygame.mouse.get_pos()
    clicked = any(pygame.mouse.get_pressed()[0] for _ in range(1))
    hover = back_button["rect"].collidepoint(mouse_pos)

    if clicked and hover and not back_button_clicked:
        if click_sound:
            click_sound.play()
        back_button_clicked = True
    elif not clicked:
        back_button_clicked = False

    draw_button(screen, back_button["rect"], back_button["color"], back_button["text"], hover, clicked and hover)

    if fade_alpha > 0:
        fade_surface = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
        fade_surface.fill((255, 255, 255, fade_alpha))
        screen.blit(fade_surface, (0, 0))

    pygame.display.flip()
    return input_rect, back_button, back_button_clicked

def draw_intro_screen(screen, background_image, character_text_image, typing_sound=None, click_sound=None):
    pygame.mixer.music.pause()
    blurred_bg = blur_surface(background_image.copy(), factor=4)
    screen.blit(blurred_bg, (0, 0))
    character_rect = character_text_image.get_rect(topleft=(50, HEIGHT - 300))
    screen.blit(character_text_image, character_rect)

    intro_text_1 = ("ยินดีต้อนรับสู่ตลาดนักเศรษฐศาสตร์รุ่นใหม่! "
                    "คุณมีเวลา 10 วันเพื่อทำเงินให้ได้ 1500 บาท "
                    "โดยเริ่มจากเงิน 1000 บาท ซื้อและขายสินค้าอย่างชาญฉลาด "
                    "มี 3 ร้าน: ข้าว, น้ำ, และเครื่องมือ "
                    "ราคาจะเปลี่ยนทุกวันตามความต้องการ "
                    "คลิกเพื่อไปร้าน, ซื้อหรือขายสินค้า, แล้วกดวันถัดไป "
                    "ระวังอย่าให้เงินหมด!")
    intro_text_2 = ("นี่คือเกมที่เกี่ยวกับหลักการเศรษฐศาสตร์มหภาค (Macroeconomics) "
                    "คุณจะได้เรียนรู้การจัดการทรัพยากรผ่านการซื้อขายสินค้า "
                    "ในตลาดที่มีการเปลี่ยนแปลงตลอดเวลา")

    stages = [
        {"text": intro_text_1, "button_text": "ต่อไป"},
        {"text": intro_text_2, "button_text": "เริ่มเกม"}
    ]
    current_stage = 0

    def split_text(text):
        words = text.split(" ")
        lines = []
        current_line = ""
        max_width = 380
        for word in words:
            test_line = current_line + word + " "
            test_surface = pygame.font.SysFont("THSarabunNew", 24).render(test_line, True, BLACK)
            if test_surface.get_width() <= max_width:
                current_line = test_line
            else:
                lines.append(current_line.strip())
                current_line = word + " "
        lines.append(current_line.strip())
        return lines

    current_lines = split_text(stages[current_stage]["text"])
    current_text = ["" for _ in current_lines]
    text_index = 0
    line_index = 0
    typing_speed = 50
    last_update = pygame.time.get_ticks()

    next_button = {"rect": pygame.Rect(WIDTH - 120, HEIGHT - 70, 100, 50), "color": ORANGE, "text": stages[current_stage]["button_text"]}
    start_game = False
    text_completed = False
    next_button_clicked = False

    clock = pygame.time.Clock()

    while not start_game:
        mouse_pos = pygame.mouse.get_pos()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                pos = event.pos
                if not text_completed:
                    if typing_sound:
                        typing_sound.stop()
                    current_text = [line for line in current_lines]
                    line_index = len(current_lines)
                    text_index = 0
                    text_completed = True
                elif next_button["rect"].collidepoint(pos):
                    if not next_button_clicked:
                        if click_sound:
                            click_sound.play()
                        next_button_clicked = True
                    if current_stage < len(stages) - 1:
                        current_stage += 1
                        current_lines = split_text(stages[current_stage]["text"])
                        current_text = ["" for _ in current_lines]
                        text_index = 0
                        line_index = 0
                        text_completed = False
                        next_button["text"] = stages[current_stage]["button_text"]
                    else:
                        start_game = True
            elif event.type == pygame.MOUSEBUTTONUP:
                next_button_clicked = False

        current_time = pygame.time.get_ticks()
        if current_time - last_update >= typing_speed and line_index < len(current_lines) and not text_completed:
            if text_index < len(current_lines[line_index]):
                current_text[line_index] = current_lines[line_index][:text_index + 1]
                if typing_sound:
                    typing_sound.play()
                text_index += 1
                last_update = current_time
            elif line_index < len(current_lines) - 1:
                line_index += 1
                text_index = 0
                last_update = current_time
            else:
                text_completed = True

        screen.blit(blurred_bg, (0, 0))
        bubble_rect = pygame.Rect(200, 120, 400, len(current_lines) * 24 + 40)
        pygame.draw.rect(screen, GRAY, bubble_rect)
        pygame.draw.rect(screen, DARK_GRAY, bubble_rect, 2)
        pygame.draw.line(screen, LIGHT_GRAY, (bubble_rect.left, bubble_rect.top), (bubble_rect.right, bubble_rect.top), 2)
        pygame.draw.line(screen, LIGHT_GRAY, (bubble_rect.left, bubble_rect.top), (bubble_rect.left, bubble_rect.bottom), 2)
        pygame.draw.line(screen, DARK_GRAY, (bubble_rect.right - 2, bubble_rect.top), (bubble_rect.right - 2, bubble_rect.bottom), 2)
        pygame.draw.line(screen, DARK_GRAY, (bubble_rect.left, bubble_rect.bottom - 2), (bubble_rect.right, bubble_rect.bottom - 2), 2)

        for i, line in enumerate(current_text):
            draw_text(screen, line, bubble_rect.centerx, bubble_rect.top + 20 + i * 24, font_size=24, color=BLACK, center=True)

        screen.blit(character_text_image, character_rect)

        if text_completed:
            hover = next_button["rect"].collidepoint(mouse_pos)
            draw_button(screen, next_button["rect"], next_button["color"], next_button["text"], hover, False)

        pygame.display.flip()
        clock.tick(60)

    pygame.mixer.music.unpause()
    return True

def draw_game_over_screen(screen, game_over_message, background_gameover=None, click_sound=None):
    screen.blit(background_gameover, (0, 0))

    draw_text(screen, game_over_message, WIDTH // 2, HEIGHT // 2 - 50, font_size=36, color=RED if "แพ้" in game_over_message else GREEN, center=True)

    buttons = [
        {"text": "กลับไปหน้าเมนู", "rect": pygame.Rect(WIDTH // 2 - 150, HEIGHT // 2 + 50, 150, 50), "color": ORANGE, "action": "return_to_menu", "key": "menu"},
        {"text": "เริ่มเกมใหม่", "rect": pygame.Rect(WIDTH // 2 + 10, HEIGHT // 2 + 50, 150, 50), "color": GREEN, "action": "restart_game", "key": "restart"},
    ]

    running = True
    action = None
    button_clicked = {"menu": False, "restart": False}

    clock = pygame.time.Clock()

    while running:
        mouse_pos = pygame.mouse.get_pos()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                pos = event.pos
                for btn in buttons:
                    if btn["rect"].collidepoint(pos):
                        if not button_clicked[btn["key"]]:
                            if click_sound:
                                click_sound.play()
                            button_clicked[btn["key"]] = True
                        action = btn["action"]
                        running = False
                        break
            elif event.type == pygame.MOUSEBUTTONUP:
                for btn in buttons:
                    button_clicked[btn["key"]] = False

        for btn in buttons:
            hover = btn["rect"].collidepoint(mouse_pos)
            draw_button(screen, btn["rect"], btn["color"], btn["text"], hover, False)

        pygame.display.flip()
        clock.tick(60)

    return action

def main(screen):
    # โหลดเสียง
    try:
        click_sound = pygame.mixer.Sound("click.wav")
    except FileNotFoundError:
        click_sound = None
    try:
        shop_sound = pygame.mixer.Sound("shop.wav")
        shop_sound.set_volume(0.2)
    except FileNotFoundError:
        shop_sound = None
    try:
        typing_sound = pygame.mixer.Sound("typing.wav")
        typing_sound.set_volume(0.3)
    except FileNotFoundError:
        typing_sound = None
    try:
        pygame.mixer.music.load("background_music.mp3")
    except FileNotFoundError:
        pass

    # โหลดทรัพยากรภาพ
    try:
        background_image = pygame.image.load("background_gameplay.png").convert()
        background_image = pygame.transform.scale(background_image, (WIDTH, HEIGHT))
    except FileNotFoundError:
        background_image = pygame.Surface((WIDTH, HEIGHT))
        background_image.fill((173, 216, 230))

    try:
        background_shop1 = pygame.image.load("background_shop1.png").convert()
        background_shop1 = pygame.transform.scale(background_shop1, (WIDTH, HEIGHT))
    except FileNotFoundError:
        background_shop1 = pygame.Surface((WIDTH, HEIGHT))
        background_shop1.fill((173, 216, 230))

    try:
        background_shop2 = pygame.image.load("background_shop2.png").convert()
        background_shop2 = pygame.transform.scale(background_shop2, (WIDTH, HEIGHT))
    except FileNotFoundError:
        background_shop2 = pygame.Surface((WIDTH, HEIGHT))
        background_shop2.fill((173, 216, 230))

    try:
        background_shop3 = pygame.image.load("background_shop3.png").convert()
        background_shop3 = pygame.transform.scale(background_shop3, (WIDTH, HEIGHT))
    except FileNotFoundError:
        background_shop3 = pygame.Surface((WIDTH, HEIGHT))
        background_shop3.fill((173, 216, 230))

    try:
        background_gameover = pygame.image.load("background_gameover.png").convert()
        background_gameover = pygame.transform.scale(background_gameover, (WIDTH, HEIGHT))
    except FileNotFoundError:
        background_gameover = pygame.Surface((WIDTH, HEIGHT))
        background_gameover.fill((173, 216, 230))

    try:
        shop1_image = pygame.image.load("shop1.png").convert_alpha()
        shop1_image = scale_image(shop1_image)
    except FileNotFoundError:
        shop1_image = pygame.Surface((150, 150))
        shop1_image.fill(GRAY)

    try:
        shop2_image = pygame.image.load("shop2.png").convert_alpha()
        shop2_image = scale_image(shop2_image)
    except FileNotFoundError:
        shop2_image = pygame.Surface((150, 150))
        shop2_image.fill(GRAY)

    try:
        shop3_image = pygame.image.load("shop3.png").convert_alpha()
        shop3_image = scale_image(shop3_image)
    except FileNotFoundError:
        shop3_image = pygame.Surface((150, 150))
        shop3_image.fill(GRAY)

    try:
        character_front_image = pygame.image.load("character.png").convert_alpha()
        character_front_image = pygame.transform.scale(character_front_image, (70, 70))
    except FileNotFoundError:
        character_front_image = pygame.Surface((70, 70))
        character_front_image.fill(BLACK)

    try:
        character_left_image = pygame.image.load("character_left.png").convert_alpha()
        character_left_image = pygame.transform.scale(character_left_image, (70, 70))
    except FileNotFoundError:
        character_left_image = pygame.Surface((70, 70))
        character_left_image.fill(BLACK)

    try:
        character_right_image = pygame.image.load("character_right.png").convert_alpha()
        character_right_image = pygame.transform.scale(character_right_image, (70, 70))
    except FileNotFoundError:
        character_right_image = pygame.Surface((70, 70))
        character_right_image.fill(BLACK)

    try:
        character_text_image = pygame.image.load("character_text.png").convert_alpha()
        character_text_image = pygame.transform.scale(character_text_image, (300, 300))
    except FileNotFoundError:
        character_text_image = pygame.Surface((300, 300))
        character_text_image.fill(BLACK)

    while True:
        # เรียกหน้าจอแนะนำหลังจากโหลดทรัพยากรแล้ว
        if not draw_intro_screen(screen, background_image, character_text_image, typing_sound, click_sound):
            return

        pygame.mixer.music.play(-1)
        game = Game("ผู้เล่น")
        character = Character(WIDTH // 2, HEIGHT // 2, character_front_image, character_left_image, character_right_image)

        running = True
        current_screen = "main"
        current_shop = None
        current_shop_item = None
        target_shop = None
        current_shop_number = None
        input_active = False
        input_amount = ""
        current_action = None
        message = ""
        last_hovered_shop = None
        fade_alpha = 0
        fade_direction = 0
        fade_start_time = 0
        next_day_clicked = False
        button_clicked = None
        back_button_clicked = False

        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    pos = event.pos
                    if current_screen == "main":
                        character.update()
                        shops, next_day_button, last_hovered_shop, next_day_clicked = draw_main_scene(screen, game, character, last_hovered_shop, fade_alpha, next_day_clicked, background_image, shop1_image, shop2_image, shop3_image, shop_sound, click_sound)
                        if next_day_button["rect"].collidepoint(pos):
                            character.rect.center = (WIDTH // 2, HEIGHT // 2)
                            character.target = None
                            fade_direction = 1
                            fade_start_time = pygame.time.get_ticks()
                            game.next_day()
                        else:
                            shop_clicked = False
                            for shop in shops:
                                if shop["rect"].collidepoint(pos):
                                    target_shop = shop
                                    character.target = shop["rect"].center
                                    shop_clicked = True
                                    break
                            if not shop_clicked:
                                character.target = pos
                                target_shop = None
                    elif current_screen == "shop":
                        buttons, button_clicked = draw_shop_menu(screen, game, current_shop, current_shop_item, current_shop_number, fade_alpha, button_clicked, background_shop1, background_shop2, background_shop3, click_sound)
                        for btn in buttons:
                            if btn["rect"].collidepoint(pos):
                                if btn["action"] == "exit_shop":
                                    current_screen = "main"
                                    current_shop = None
                                    current_shop_item = None
                                    current_shop_number = None
                                    target_shop = None
                                else:
                                    current_screen = "input"
                                    current_action = btn["action"]
                                    message = f"กรุณาใส่จำนวนสำหรับ {btn['text']}: "
                                    input_amount = ""
                                    input_active = False
                    elif current_screen == "input":
                        input_rect, back_button, back_button_clicked = draw_input_menu(screen, message, input_amount, input_active, fade_alpha, back_button_clicked, background_image, click_sound)
                        if input_rect.collidepoint(pos):
                            input_active = True
                        elif back_button["rect"].collidepoint(pos):
                            current_screen = "shop"
                            input_amount = ""
                            current_action = None
                            input_active = False
                            message = ""
                        else:
                            input_active = False

                if current_screen == "input" and input_active:
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_RETURN:
                            amount = int(input_amount) if input_amount.isdigit() else 0
                            if current_action == "buy_rice":
                                game.message = game.player.buy("rice", amount, game.market)
                            elif current_action == "buy_water":
                                game.message = game.player.buy("water", amount, game.market)
                            elif current_action == "buy_tools":
                                game.message = game.player.buy("tools", amount, game.market)
                            elif current_action == "sell_rice":
                                game.message = game.player.sell("rice", amount, game.market)
                            elif current_action == "sell_water":
                                game.message = game.player.sell("water", amount, game.market)
                            elif current_action == "sell_tools":
                                game.message = game.player.sell("tools", amount, game.market)
                            input_active = False
                            current_screen = "shop"
                            input_amount = ""
                            current_action = None
                        elif event.key == pygame.K_BACKSPACE:
                            input_amount = input_amount[:-1]
                        elif event.unicode.isdigit():
                            input_amount += event.unicode

            current_time = pygame.time.get_ticks()
            if fade_direction != 0:
                elapsed_time = current_time - fade_start_time
                if elapsed_time < 250:
                    fade_alpha = int(255 * (elapsed_time / 250))
                elif elapsed_time < 500:
                    fade_alpha = int(255 * (1 - (elapsed_time - 250) / 250))
                else:
                    fade_alpha = 0
                    fade_direction = 0
                    fade_start_time = 0

            if current_screen == "main":
                character.update()
                shops, next_day_button, last_hovered_shop, next_day_clicked = draw_main_scene(screen, game, character, last_hovered_shop, fade_alpha, next_day_clicked, background_image, shop1_image, shop2_image, shop3_image, shop_sound, click_sound)
                if target_shop and character.target is None:
                    current_screen = "shop"
                    current_shop = target_shop["name"]
                    current_shop_item = target_shop["item"]
                    current_shop_number = target_shop["number"]
                    target_shop = None
            elif current_screen == "shop":
                buttons, button_clicked = draw_shop_menu(screen, game, current_shop, current_shop_item, current_shop_number, fade_alpha, button_clicked, background_shop1, background_shop2, background_shop3, click_sound)
            elif current_screen == "input":
                input_rect, back_button, back_button_clicked = draw_input_menu(screen, message, input_amount, input_active, fade_alpha, back_button_clicked, background_image, click_sound)

            game_over = game.check_game_over()
            if game_over:
                pygame.mixer.music.stop()
                action = draw_game_over_screen(screen, game_over, background_gameover, click_sound)
                if action == "return_to_menu":
                    if main_menu(screen, click_sound):
                        running = False
                    else:
                        pygame.quit()
                        sys.exit()
                elif action == "restart_game":
                    running = False

    pygame.quit()

if __name__ == "__main__":
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    if main_menu(screen):
        main(screen)
    else:
        pygame.quit()
        sys.exit()